In [22]:
import sqlite3
import pandas as pd



In [23]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')
c = conn.cursor()

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [27]:
query10=("""
SELECT facility_name, sum(cost) AS revenue
FROM Members AS m
INNER JOIN 
    
    (SELECT b.memid,b.starttime,f.name as facility_name, 
        CASE
    	WHEN b.memid=0 THEN slots*guestcost
    	ELSE slots*membercost END AS cost
        FROM Bookings as b
            INNER JOIN Facilities as f
            USING (facid)
            ) AS bf
ON m.memid=bf.memid
GROUP BY facility_name
HAVING revenue<1000
ORDER BY revenue DESC
""")

c.execute(query10)
df=pd.DataFrame(c.fetchall())
col=[ row[0] for row in c.description]
df.columns=col
df

,facility_name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */



In [28]:
query11=("""
SELECT m1.surname,m1.firstname, m1.recommendedby,  m2.surname AS recommendedby_surname, m2.firstname AS recommendedby_firstname
FROM 
     (SELECT *
     FROM Members as m
     WHERE m.recommendedby !="") AS m1
INNER JOIN Members AS m2
WHERE m1.recommendedby=m2.memid
ORDER BY m1.surname,m1.firstname
""")

c.execute(query11)
df=pd.DataFrame(c.fetchall())
col=[ row[0] for row in c.description]
df.columns=col
df

,surname,firstname,recommendedby,recommendedby_surname,recommendedby_firstname
0,Bader,Florence,9,Stibbons,Ponder
1,Baker,Anne,9,Stibbons,Ponder
2,Baker,Timothy,13,Farrell,Jemima
3,Boothe,Tim,3,Rownam,Tim
4,Butters,Gerald,1,Smith,Darren
5,Coplin,Joan,16,Baker,Timothy
6,Crumpet,Erica,2,Smith,Tracy
7,Dare,Nancy,4,Joplette,Janice
8,Genting,Matthew,5,Butters,Gerald
9,Hunt,John,30,Purview,Millicent


/* Q12: Find the facilities with their usage by member, but not guests */

In [33]:
query12=("""

SELECT f.name AS facility_name, m.surname,COUNT(slots)
FROM Bookings AS b, Facilities AS f, Members AS m
WHERE b.facid=f.facid AND b.memid!=0 AND b.memid=m.memid
	
GROUP BY f.name,m.surname

""")
c.execute(query12)
df=pd.DataFrame(c.fetchall())
col=[ row[0] for row in c.description]
df.columns=col
df

,facility_name,surname,COUNT(slots)
0,Badminton Court,Bader,9
1,Badminton Court,Baker,17
2,Badminton Court,Boothe,12
3,Badminton Court,Butters,20
4,Badminton Court,Crumpet,2
...,...,...,...
166,Tennis Court 2,Rumney,1
167,Tennis Court 2,Sarwin,11
168,Tennis Court 2,Smith,22
169,Tennis Court 2,Stibbons,31


/* Q13: Find the facilities usage by month, but not guests */

In [35]:
query13=("""SELECT f.name AS facility_name, SUBSTRING(starttime,6,2) as month,COUNT(slots)
FROM Bookings AS b, Facilities AS f
WHERE b.facid=f.facid AND b.memid!=0 
GROUP BY month,facility_name""")

c.execute(query13)
df=pd.DataFrame(c.fetchall())
col=[ row[0] for row in c.description]
df.columns=col
df

,facility_name,month,COUNT(slots)
0,Badminton Court,07,51
1,Massage Room 1,07,77
2,Massage Room 2,07,4
3,Pool Table,07,103
4,Snooker Table,07,68
5,Squash Court,07,23
6,Table Tennis,07,48
7,Tennis Court 1,07,65
8,Tennis Court 2,07,41
9,Badminton Court,08,132
